# dataforgood waterlooregion
Investigating the impact of Twitter on local non-profits

### Twitter is a means to community

### Focus on
* is_retweet = False
* retweet_count

In [1]:
import pandas as pd
import matplotlib_inline as plt
tweets_raw = pd.read_csv('tweets_all_revised2.csv', low_memory=False)
organization_raw = pd.read_csv('organization_profiles.csv', index_col = [0])
df_raw = organization_raw.merge(tweets_raw, on=['screen_name'])

df = df_raw[df_raw['is_retweet']==False]

### Quick Overview of our Focus

In [2]:
df_glance = df[['screen_name', 'retweet_count']]
df_glance = df_glance.groupby('screen_name').describe(percentiles=[0.5, 0.75, 0.85, 0.9, 0.95])
df_glance.sort_values([('retweet_count', 'max')], ascending=False)

retweet_count                                               \
                        count      mean       std  min  50%  75%  85%  90%   
screen_name                                                                  
KidsAbility            1969.0  3.122397  7.760041  0.0  2.0  4.0  5.0  7.0   
CambFoodBank           2344.0  2.133532  3.865163  0.0  1.0  3.0  4.0  5.0   
KitchLibrary           2621.0  2.081267  5.347672  0.0  1.0  2.0  4.0  5.0   
SASCWR                 1975.0  2.757468  5.382757  0.0  1.0  3.0  5.0  7.0   
Nutrition4Learn        2302.0  3.452215  7.653918  0.0  1.0  3.0  6.0  9.0   
...                       ...       ...       ...  ...  ...  ...  ...  ...   
Travind111              282.0  0.574468  0.878644  0.0  0.0  1.0  1.0  2.0   
LINKPicnic              180.0  0.366667  0.783204  0.0  0.0  0.0  1.0  1.0   
PCL_KW                  183.0  0.065574  0.357134  0.0  0.0  0.0  0.0  0.0   
family_kind             192.0  0.093750  0.371163  0.0  0.0  0.0  0.0  0.0   
KW_Chamber              102.0  0.098039  0.298836  0.0  0.0  0.0  0.0  0.0   

                              
                  95%    max  
screen_name                   
KidsAbility       9.0  220.0  
CambFoodBank      7.0  144.0  
KitchLibrary      8.0  142.0  
SASCWR           10.0  130.0  
Nutrition4Learn  16.0  111.0  
...               ...    ...  
Travind111        2.0    5.0  
LINKPicnic        2.0    5.0  
PCL_KW            0.0    4.0  
family_kind       1.0    2.0  
KW_Chamber        1.0    1.0  

[86 rows x 10 columns]

### Ridge Regression Matrix of TF-IDF features from Tweet Text

In [3]:
#parse text
X_text = df
X = X_text['text_clean'].dropna()

#return corresponding target labels
y_text = tweets_raw.loc[X.index]
y = y_text['retweet_count']

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

vect = TfidfVectorizer()
ridge = Ridge()


pipe = Pipeline([
    ('vect', vect),
    ('model', ridge)
])
pipe.fit(X_train, y_train)

retweet_weighted_word = list(zip(pipe['vect'].get_feature_names(), pipe['model'].coef_ ))
from operator import itemgetter
sorted(retweet_weighted_word, key = itemgetter(1), reverse=True)


[('footwear', 29.90771096377448),
 ('abusers', 23.81612816493302),
 ('drunk', 22.431061585301396),
 ('enha', 21.82280368737374),
 ('ruling', 20.86878666775963),
 ('defence', 20.472345713710716),
 ('rapists', 20.05451156749657),
 ('tanyatalaga', 19.92503244541511),
 ('listeni', 19.320387970122212),
 ('festivaloflivingstreets', 17.656592574813203),
 ('ryerson', 17.488917742328567),
 ('blanketsneeded', 17.178179690485724),
 ('snack', 16.662254931963364),
 ('highprofile', 15.17389297130515),
 ('coronation', 14.48128377104493),
 ('solslib', 14.438290417920163),
 ('locations', 14.410869879989214),
 ('longs', 14.182462802945311),
 ('themoreyouknow', 14.080751500495467),
 ('cloth', 13.476506100976385),
 ('assaulted', 13.080643895162506),
 ('colourful', 12.2620738889034),
 ('decolonize', 11.991273925519465),
 ('gothic', 11.941363258593148),
 ('elgin', 11.78815147091612),
 ('socks', 11.75988684995414),
 ('slurring', 11.633999068466574),
 ('stomper', 11.628292044555158),
 ('counselloradvocate', 1

In [5]:
df[df['text_clean'].str.contains("abusers", case=False, na = False)].sort_values("retweet_count", ascending=False) #retweet false
# df_raw[df_raw['text_clean'].str.contains("abusers", case=False, na = False)].sort_values("retweet_count", ascending=False) #retweet true/false

,Organization Name_x,twitter_handle,screen_name,agency_catergory,followers_count,friends_count,location,tweet_id,user_id,created_at,...,favorite_count,retweet_count,URL,hashtags,has_media,is_quote,is_reply,is_retweet,Organization Name_y,text_clean
133618,Sexual Assault Support Centre of Waterloo Region,@SASCWR,SASCWR,Community and Social Services,2967,1324,"Waterloo Region, Ontario",133622,247092883,2020-06-04 17:30:26,...,102.0,130.0,https://twitter.com/i/web/status/1268595983287...,[],False,False,False,False,Sexual Assault Support Centre of Waterloo Region,This ruling means that rapists and abusers can...
163577,Women's Crisis Services of Waterloo Region,@WomensCrisisSWR,WomensCrisisSWR,"Children, Youth and Family",4789,2822,Waterloo Region,163581,185319351,2020-04-29 12:47:25,...,8.0,7.0,https://twitter.com/i/web/status/1255478795911...,[],False,False,False,False,Women's Crisis Services of Waterloo Region,Fewer women seeking shelter from abusers a wor...
163321,Women's Crisis Services of Waterloo Region,@WomensCrisisSWR,WomensCrisisSWR,"Children, Youth and Family",4789,2822,Waterloo Region,163325,185319351,2020-08-28 13:00:00,...,8.0,3.0,https://twitter.com/i/web/status/1299330894981...,[],False,False,False,False,Women's Crisis Services of Waterloo Region,The fact that so many attackers had been domes...
163986,Women's Crisis Services of Waterloo Region,@WomensCrisisSWR,WomensCrisisSWR,"Children, Youth and Family",4789,2822,Waterloo Region,163990,185319351,2019-10-18 13:00:01,...,2.0,3.0,https://www.psychologytoday.com/us/blog/why-ba...,['thisisDV'],False,False,False,False,Women's Crisis Services of Waterloo Region,Abusers Use Technology to Stalk Scare and Scam...
163309,Women's Crisis Services of Waterloo Region,@WomensCrisisSWR,WomensCrisisSWR,"Children, Youth and Family",4789,2822,Waterloo Region,163313,185319351,2020-09-06 14:30:00,...,6.0,1.0,https://twitter.com/i/web/status/1302615035420...,[],False,False,False,False,Women's Crisis Services of Waterloo Region,“Abuse can take many forms advocates say Often...
136210,Shelter Movers Waterloo Region,@ShelterMovers,ShelterMovers,Community and Social Services,1335,1105,Canada,136214,725859947322433536,2021-08-20 16:01:29,...,2.0,0.0,https://twitter.com/i/web/status/1428749033221...,[],False,False,False,False,Shelter Movers Waterloo Region,Abusers use coercive control tactics to contro...
166070,YW Kitchener-Waterloo,@ywcakw,ywcakw,"Children, Youth and Family",3410,2178,"Kitchener, Ontario",166074,219338313,2020-04-30 15:46:11,...,1.0,0.0,https://twitter.com/i/web/status/1255886173312...,[],False,False,False,False,YW Kitchener-Waterloo,While were all doing our best to flatten the c...


### Qualitative Thoughts:
    
 In this sample, many of the words at the top of the list are related to **Organizations that work within Sexual Support/Education**

* Think about the **embodiment of the shared values** and needs **of the members** involved

 
* Words like: Drunk/ Slurring/Abusers
    * They are **informative posts** that can have emotional ties to **personal stories** and **government policies** that affect one's sexual wellbeing.
    
***This is how they engage with their members.***
    
As an organization, it is important to invision your community in order to correctly:
1. Find/discover new audiences / Audiences discover your organization
2. Engagement with those people
3. Eventually membership and becoming part of the community

"What kind of membership are you offering people?"

### FF Other Exploration Ideas:

##### Viral Popoff
Hypothesis: few tweets going viral that quickly/**exponentially get retweeted due to reaching 2nd, 3rd level networks** that in turn **signal a potential audience universe that is engaging and active.**

- organization top 20 percentile of largest Standard Deviations
- retweets vs follower_count (if retweet > followers)





##### Consistent Engagement
Hypothesis: **Consistency of posts/events** lead to **more likely increase in retweets**/following growth, due to community building upon encounters, **sense of belonging**

- Deviation of # of usual posts (ex, weekly, monthly - as long as it is regular)